# Versão 1.1

Nessa versão utilizei K-means para os 6 clusteres, dados normalizados somente por feature e NÃO utilizei o filtro Kalman.

Coluna 'cycle_num' dos dados forma normalizados ( value / dataset.['cycle_num'].max() ) e depois multiplicando pelo valor novamente ao avaliar na métrica da competição. (melhora de 8%)

Outra modificação foi a inclusão desta coluna no treinamento do modelo. O resultado foi absurdamente melhorado. (imensurável a melhora no modelo)

In [1]:
#Importando as bibliotecas

# Data Analysis Tools
import pandas as pd
import numpy as np
from time import time
import logging as log

# Visualization Tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display

# Machine Learning Tools
import sklearn
import sklearn.mixture as mix
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn import preprocessing
import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import ShuffleSplit
import pickle


#Scoreing tools
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
# from sklearn.metrics import mean_squared_log_error

# plt.rcParams['figure.figsize'] = (10, 10)

def get_sensor_names(tag_names):
    """ Get tagnames starting with sensor.
    
    :param tag_names: Input time series data frame
    
    :return list of string tag names starting with sensor.
    """
    
    return [tag_name for tag_name in tag_names if tag_name.startswith('sensor')]

c:\programdata\anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\programdata\anaconda2\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Import dos dados

#Dataset de treino
train_final_data = pd.read_csv('train_final_data.csv', sep=',')
target_final_data_train_scaled = pd.read_csv('target_final_data_train_scaled.csv', sep=',', names = ['cycle_num'],header = None)
target_final_data_train = pd.read_csv('target_final_data_train.csv', sep=',', names = ['cycle_num'],header = None)

#Dataset de teste
test_final_data = pd.read_csv('test_final_data.csv', sep=',')
target_final_data_test_scaled = pd.read_csv('target_final_data_test_scaled.csv', sep=',', names = ['cycle_num'],header = None)
target_final_data_test = pd.read_csv('target_final_data_test.csv', sep=',', names = ['cycle_num'],header = None)

#Dataset de teste final
finaltest_final_data = pd.read_csv('finaltest_final_data.csv', sep=',')
target_final_data_finaltest_scaled = pd.read_csv('target_final_data_finaltest_scaled.csv', sep=',', names = ['cycle_num'],header = None)
target_final_data_finaltest = pd.read_csv('target_final_data_finaltest.csv', sep=',', names = ['cycle_num'],header = None)


print "Os dados dos sensores e settings:"
print "O dataset de treino tem {} linhas de dados e {} atributos.".format(*train_final_data.shape)
print "O dataset de teste tem {} linhas de dados e {} atributos.".format(*test_final_data.shape)
print "O dataset de testes finais tem {} linhas de dados e {} atributos.".format(*finaltest_final_data.shape)

print "\n"

print "Os dados dos sensores e settings:"
print "O dataset target normalizado de treino tem {} linhas de dados e {} atributos.".format(*target_final_data_train_scaled.shape)
print "O dataset target normalizado de teste tem {} linhas de dados e {} atributos.".format(*target_final_data_test_scaled.shape)
print "O dataset target normalizado de testes finais tem {} linhas de dados e {} atributos.".format(*target_final_data_finaltest_scaled.shape)

print "\n"

print "Os dados dos sensores e settings:"
print "O dataset target de treino tem {} linhas de dados e {} atributos.".format(*target_final_data_train.shape)
print "O dataset target de teste tem {} linhas de dados e {} atributos.".format(*target_final_data_test.shape)
print "O dataset target de testes finais tem {} linhas de dados e {} atributos.".format(*target_final_data_finaltest.shape)


Os dados dos sensores e settings:
O dataset de treino tem 45918 linhas de dados e 19 atributos.
O dataset de teste tem 29820 linhas de dados e 19 atributos.
O dataset de testes finais tem 55156 linhas de dados e 19 atributos.


Os dados dos sensores e settings:
O dataset target normalizado de treino tem 45918 linhas de dados e 1 atributos.
O dataset target normalizado de teste tem 29820 linhas de dados e 1 atributos.
O dataset target normalizado de testes finais tem 55156 linhas de dados e 1 atributos.


Os dados dos sensores e settings:
O dataset target de treino tem 45918 linhas de dados e 1 atributos.
O dataset target de teste tem 29820 linhas de dados e 1 atributos.
O dataset target de testes finais tem 55156 linhas de dados e 1 atributos.


In [19]:
target_final_data_train.max()

cycle_num    357
dtype: int64

# Modelos de regressão

In [ ]:
# #ATENÇÃO------------->>>>>>>>>>> "Teste com o cross_val_predict para verificar os dados? Entender o que essa função realiza"
# from xgboost import XGBRegressor as xgb
# lr = linear_model.LinearRegression()
# # lr = xgb(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7)
# boston = train_final_data
# y = target_final_data

# # cross_val_predict returns an array of the same size as `y` where each entry
# # is a prediction obtained by cross validation:
# predicted = cross_val_predict(lr, train_final_data, target_final_data, cv=100)

# fig, ax = plt.subplots(figsize=(10,10))
# plt.title(lr.__class__.__name__,fontsize=30)
# ax.scatter(y, predicted, c=('b', 'k'))
# ax.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=4)
# ax.set_xlabel('Measured',fontsize=25)
# ax.set_ylabel('Predicted',fontsize=25)
# ax.tick_params(labelsize=20)
# plt.savefig(lr.__class__.__name__+'_Modelo.png', format='png', dpi=100)
# plt.show()

In [3]:
#Funções para regressão dos dados:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
     
    clf.fit(X_train, y_train)
    print "O modelo foi treinado!"
    return 

def predict_labels_train(clf, X, y):
    ''' Faz uma estimativa utilizando um modelo de regressão ajustado em algum score ainda a ser definido. '''
       
    y_pred = clf.predict(X)
    
    
    # Imprime os resultados de retorno
    print ("Pronto!\n As previsões de treino para o modelo {} foram feitas!".format(clf.__class__.__name__))
    return  {"R2 score": r2_score(y.values, y_pred), 
             "Explained Variance Score": explained_variance_score(y.values, y_pred),
             "Mean Squared Error": mean_squared_error(y.values, y_pred)
            }

def predict_labels_test(clf, X, y):
    ''' Faz uma estimativa utilizando um modelo de regressão ajustado em algum score ainda a ser definido. '''
       
    y_pred = clf.predict(X)

    
    
    
    # Imprime os resultados de retorno
    print ("Pronto!\n As previsões de teste para o modelo {} foram feitas!\n".format(clf.__class__.__name__))
    return  {"R2 score": r2_score(y.values, y_pred), 
             "Explained Variance Score": explained_variance_score(y.values, y_pred),
             "Mean Squared Error": mean_squared_error(y.values, y_pred)
                          }

# def metrica(d,a1,a2)

#     for i, x in enumerate(d):
#         if (x < 0):
#             np.exp(-(d/a1))
#         else:
#             np.exp(d/a2)

In [11]:
# Neste passo irei realizar os modelos de regressão e mostrar os resultados

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn import linear_model
from xgboost import XGBRegressor as xgb

Regression = [DecisionTreeRegressor(),SVR(C=1.0, epsilon=0.2) ,KNeighborsRegressor(n_neighbors=2,algorithm = 'auto'),linear_model.LinearRegression(),
              xgb(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=7) 
             ]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]
            }

################################DADOS PASSADOS NO MODELO###########################################################
# ##Treino
# #Coluna targe do dataset
# target_final_data_train = train_final_data['cycle_num']
# #Colunas com os dados
# train_final_data = train_final_data.drop(['cycle_num'], axis = 1)

##################################################################################################################

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])
                
                
Results = pd.DataFrame(resultados)
Results = Results[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results.head()


Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo DecisionTreeRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo DecisionTreeRegressor foram feitas!

Próximo modelo...



c:\programdata\anaconda2\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo SVR foram feitas!
Pronto!
 As previsões de teste para o modelo SVR foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo KNeighborsRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo KNeighborsRegressor foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo LinearRegression foram feitas!
Pronto!
 As previsões de teste para o modelo LinearRegression foram feitas!

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,DecisionTreeRegressor,1.000000,1.000000,4.988344e-36,0.606471,0.606534,0.014192,"(32142, 19)","(13776, 19)"
1,SVR,0.726076,0.728013,1.013936e-02,0.721880,0.723321,0.010030,"(32142, 19)","(13776, 19)"
2,KNeighborsRegressor,0.925185,0.925224,2.769287e-03,0.771350,0.771444,0.008246,"(32142, 19)","(13776, 19)"
3,LinearRegression,0.759268,0.759268,8.910753e-03,0.758689,0.758740,0.008703,"(32142, 19)","(13776, 19)"
4,XGBRegressor,0.873570,0.873570,4.679826e-03,0.831642,0.831659,0.006072,"(32142, 19)","(13776, 19)"


# Modelo escolhido

# XGBOOST

Utilizando o XGboost para uma primeira rodada de previsão

In [13]:
from xgboost import XGBRegressor as xgb

Regression = [xgb(n_estimators=150, learning_rate=0.05, gamma=0, subsample=0.75,colsample_bytree=0.7, 
                  max_depth=7)]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]
            }

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])

                  
Results_xgboost = pd.DataFrame(resultados)
Results_xgboost = Results_xgboost[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results_xgboost.head()

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,XGBRegressor,0.86927,0.869271,0.004839,0.832437,0.832454,0.006043,"(32142, 19)","(13776, 19)"


# Ajustando o modelo do Xgboost

In [32]:
def performance_metric(y_true, y_predict):
    """ Calcular e retornar a pontuação de desempenho entre 
        valores reais e estimados baseado na métrica escolhida. """
    # TODO: Calcular a pontuação de desempenho entre 'y_true' e 'y_predict'
    score = r2_score(y_true, y_predict)
#     explainedvariancescore =  explained_variance_score(y_true, y_predict)
#     meansquarederror = mean_squared_error(y_true, y_predict)
          
    
    # Devolver a pontuação
    return score

In [33]:
def fit_model(X, y):

    from xgboost import XGBRegressor as xgb
    cv_sets = ShuffleSplit(X.shape[0], n_iter = 10, test_size = 0.2, random_state = 1)
#     cv_sets = train_test_split(X, y, train_size=0.7 , random_state = 1)
    estimator = xgb()
   
    # TODO: Gerar um dicionário para o parâmetro 'max_depth' com um alcance de 1 a 10
    Params = {'n_estimators':[100,150,200], 
          'learning_rate':[0.05,0.07,0.4],
          'colsample_bytree':[0.5,0.7,1],
          'max_depth':[5,7,10]
         }

    # TODO: Transformar 'performance_metric' em uma função de pontuação utilizando 'make_scorer' 
    scoring_fnc = make_scorer(performance_metric)
    

    # TODO: Gerar o objeto de busca em matriz
    grid = GridSearchCV( estimator=estimator, param_grid=Params, scoring=scoring_fnc, cv=cv_sets)
    #grid = GridSearchCV( estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets2)

    # Ajustar o objeto de busca em matriz com os dados para calcular o modelo ótimo
    grid = grid.fit(X, y)
   

    # Devolver o modelo ótimo depois de realizar o ajuste dos dados
    return grid.best_estimator_

print ('Tudo ok até então!')

Tudo ok até então!


In [35]:
reg = fit_model(train_final_data, target_final_data_train_scaled)

In [36]:
# Produzir valores para 'max_depth'
print "O parâmetro 'n_estimators' é {} para o modelo ótimo.".format(reg.get_params()['n_estimators'])
print "O parâmetro 'learning_rate' é {} para o modelo ótimo.".format(reg.get_params()['learning_rate'])
print "O parâmetro 'colsample_bytree' é {} para o modelo ótimo.".format(reg.get_params()['colsample_bytree'])
print "O parâmetro 'max_depth' é {} para o modelo ótimo.".format(reg.get_params()['max_depth'])

O parâmetro 'n_estimators' é 150 para o modelo ótimo.
O parâmetro 'learning_rate' é 0.05 para o modelo ótimo.
O parâmetro 'colsample_bytree' é 0.7 para o modelo ótimo.
O parâmetro 'max_depth' é 7 para o modelo ótimo.


# Modelo Final XGBOOST

In [14]:
#Dados Finais

X_train, X_test, y_train, y_test = train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)

In [15]:
from xgboost import XGBRegressor as xgb

Regression = [xgb(n_estimators=150, learning_rate=0.05, gamma=0, subsample=0.75,colsample_bytree=0.7, 
                  max_depth=7)]

resultados = { 
        'Modelo utilizado': [],
        'Conjunto de Treino': [], 
        'Conjunto de Teste': [],
        'Pontuação R2 (treinamento)': [], 
        'Pontuação R2 (teste)': [],
        'Explained Variance Score (treinamento)':[],
        'Explained Variance Score (teste)':[],
        'Mean Squared Error (treinamento)':[],
        'Mean Squared Error (teste)':[]
            }

Dados_normalizados = [train_test_split(train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)]

for clf in Regression:
    for split in Dados_normalizados:
        
        print ("Próximo modelo...\n")
        X_train, X_test, y_train, y_test = split
        
        time_train = train_classifier(clf, X_train, y_train)
        
        time_predict_train = predict_labels_train(clf, X_train, y_train)
        
        time_predict_test = predict_labels_test(clf, X_test, y_test)
        
        resultados['Modelo utilizado'].append(clf.__class__.__name__)
        resultados['Conjunto de Treino'].append(X_train.shape)
        resultados['Conjunto de Teste'].append(X_test.shape)
        resultados['Pontuação R2 (treinamento)'].append(time_predict_train["R2 score"])
        resultados['Pontuação R2 (teste)'].append(time_predict_test["R2 score"])
        resultados['Explained Variance Score (treinamento)'].append(time_predict_train["Explained Variance Score"])
        resultados['Explained Variance Score (teste)'].append(time_predict_test["Explained Variance Score"])
        resultados['Mean Squared Error (treinamento)'].append(time_predict_train["Mean Squared Error"])
        resultados['Mean Squared Error (teste)'].append(time_predict_test["Mean Squared Error"])

                  
Results_xgboost = pd.DataFrame(resultados)
Results_xgboost = Results_xgboost[['Modelo utilizado','Pontuação R2 (treinamento)','Explained Variance Score (treinamento)','Mean Squared Error (treinamento)',
                  'Pontuação R2 (teste)','Explained Variance Score (teste)','Mean Squared Error (teste)','Conjunto de Treino','Conjunto de Teste']]
Results_xgboost.head()

Próximo modelo...

O modelo foi treinado!
Pronto!
 As previsões de treino para o modelo XGBRegressor foram feitas!
Pronto!
 As previsões de teste para o modelo XGBRegressor foram feitas!



,Modelo utilizado,Pontuação R2 (treinamento),Explained Variance Score (treinamento),Mean Squared Error (treinamento),Pontuação R2 (teste),Explained Variance Score (teste),Mean Squared Error (teste),Conjunto de Treino,Conjunto de Teste
0,XGBRegressor,0.86927,0.869271,0.004839,0.832437,0.832454,0.006043,"(32142, 19)","(13776, 19)"


In [6]:
# Params = (n_estimators = 200, 
#          learning_rate = 0.4,
#          colsample_bytree = 0.7,
#          max_depth = 5)
X_train, X_test, y_train, y_test = train_test_split(
        train_final_data, target_final_data_train_scaled, train_size=0.7 , random_state = 1)

from xgboost import XGBRegressor as xgb        

model = xgb(n_estimators = 200,learning_rate = 0.4,colsample_bytree = 0.7,max_depth = 5)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * 357 , columns = ['y_pred'], index = X_test_df.index)

#Saving the model
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [7]:
data_colunas = [y_pred_final,target_final_data_train,X_test_df]
# target_final_data_train

engine_train = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_train = engine_train.dropna(axis = 0, how = 'any')

engine_train.tail()

,y_pred,cycle_num,unit,overall setting,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,...,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor20,"sensor9,14_break"
45902,118.612587,118,218.0,1.0,0.609907,0.653712,0.637759,1.0,0.546479,0.079365,...,0.0,0.660000,0.344444,0.068452,0.060274,0.791525,0.0,0.4,0.394737,0.0
45903,114.706833,119,218.0,3.0,0.701220,0.662026,0.677512,1.0,0.179361,0.660714,...,0.0,0.611842,0.313043,0.620690,0.074997,0.780723,0.0,0.8,0.412371,0.0
45905,125.770149,121,218.0,3.0,0.664634,0.521784,0.713110,1.0,0.280098,0.642857,...,0.0,0.776316,0.223188,0.775862,0.086759,0.693976,0.0,0.7,0.484536,0.0
45907,122.569839,123,218.0,1.0,0.702786,0.797999,0.766746,0.0,0.281690,0.069841,...,0.0,0.693333,0.322222,0.077381,0.037352,0.648127,0.0,0.6,0.315789,0.0
45913,129.345642,129,218.0,1.0,0.848297,0.688155,0.837719,1.0,0.343662,0.047619,...,0.0,0.773333,0.359259,0.056548,0.067462,0.799849,0.0,0.6,0.447368,0.0


In [8]:
d = []

for index, row in engine_train.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_train['d'] = d

engine_train['d'].head()

3     44.042896
8     12.786253
10    26.987179
11    32.777664
19    13.101967
Name: d, dtype: float64

In [9]:
def metrica(d):
    
    aux = 0
    for i in d:
        
        if (i < 0):
            aux = aux + (np.exp((-(i/13)))-1)           
        else:
            aux = aux + (np.exp(((i/10)))-1)
#         print aux
    return aux  

# Resultado Final

In [10]:
result = metrica(engine_train['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

O resultado final para o set de treino foi de 556854.874606 pontos.


# Test Data

In [11]:
# load the model from disk

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(test_final_data, target_final_data_test_scaled)

In [14]:
data_colunas = [y_pred_final,target_final_data_test,X_test_df]
# target_final_data_test

engine_test = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_test = engine_test.dropna(axis = 0, how = 'any')

engine_test.head()

,y_pred,cycle_num,unit,overall setting,sensor2,sensor3,sensor4,sensor6,sensor7,sensor8,...,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,sensor17,sensor20,"sensor9,14_break"
3,48.042896,4.0,1.0,0.0,0.300000,0.511334,0.339116,0.0,0.586124,0.030303,...,1.0,0.281690,0.540625,0.093750,0.118484,0.361931,0.0,0.300000,0.516129,0.0
8,21.786253,9.0,1.0,2.0,0.443730,0.339058,0.239740,1.0,0.679577,0.409091,...,0.0,0.248276,0.615894,0.377778,0.164305,0.269324,0.0,0.400000,0.706897,0.0
10,37.987179,11.0,1.0,5.0,0.666667,0.381229,0.468777,0.0,0.661392,0.142384,...,1.0,0.328467,0.952790,0.166134,0.108353,0.340909,0.0,0.444444,0.519481,0.0
11,44.777664,12.0,1.0,4.0,0.457516,0.293607,0.410168,0.0,0.666667,0.159609,...,0.0,0.328767,0.659420,0.149390,0.151310,0.257727,0.0,0.555556,0.322222,0.0
19,33.101967,20.0,1.0,2.0,0.318328,0.226503,0.247282,1.0,0.593310,0.386364,...,0.0,0.303448,0.593819,0.333333,0.161543,0.318438,0.0,0.500000,0.689655,0.0


In [15]:
d = []

for index, row in engine_test.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_test['d'] = d

engine_test['d'].head()

3     44.042896
8     12.786253
10    26.987179
11    32.777664
19    13.101967
Name: d, dtype: float64

In [16]:
result = metrica(engine_test['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

O resultado final para o set de treino foi de 331170.780303 pontos.


# Análise final com o dataset 'final_test'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(finaltest_final_data, target_final_data_finaltest_scaled, 
                                                    train_size=0 , random_state = 1)

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

y_pred = loaded_model.predict(X_test)

X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_finaltest.max(), columns = ['y_pred'], index = X_test.index)

print(result)

In [ ]:
data_colunas = [y_pred_final,X_test_df]
# target_final_data_test

engine_finaltest = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_finaltest = engine_finaltest.dropna(axis = 0, how = 'any')

engine_finaltest.head()

In [ ]:
d = []

for index, row in engine_finaltest.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_finaltest['d'] = d

engine_finaltest['d'].head()

In [ ]:
result = metrica(engine_finaltest['d'])

print ('O resultado final para o set de treino foi de {} pontos.'.format(result))

In [ ]:
X_test_df = pd.DataFrame(X_test)
y_pred_final = pd.DataFrame(y_pred * target_final_data_finaltest.max(), columns = ['y_pred'], index = X_test.index)
d = []

data_colunas = [y_pred_final,X_test_df]
# target_final_data_test

engine_finaltest = pd.concat(data_colunas, axis = 1, ignore_index=False,verify_integrity=True)

engine_finaltest = engine_finaltest.dropna(axis = 0, how = 'any')

for index, row in engine_finaltest.iterrows():
     d.append(row.y_pred - row.cycle_num)
        
engine_finaltest['d'] = d

engine_finaltest['d'].head()